<a href="https://colab.research.google.com/github/paulalqy/Algoritmos-Troll/blob/main/Online_Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Despliegue de modelos para inferencia online

En esta primera práctica vamos a aprender cómo desplegar un modelo para realizar inferencias online usando un microservicio. Para ello, utilizaremos Google Cloud Platform (GCP).

El objetivo de esta práctica es dado un modelo ya entrenado, construir un microservicio capaz de disponibilizar nuestro modelo a gran escala para peticiones en tiempo real.

El modelo lo tendremos almacenado en Google Cloud Storage y generaremos nuestro microservicio usando FastAPI y lo desplegaremos un el servicio Serverless de GCP, Cloud Run.

Al terminar esta práctica seremos capaces de crear microservicios de Machine Learning para poner en inferencia online los modelos que deseemos a gran escala.

![online_diagram](https://drive.google.com/uc?export=view&id=1aKmkzTqp0hG1VyuUIWSVbxQa7UKYEo9T)

# Para empezar... ¿Qué es una API?

Para construir aplicaciones que sean escalables e interactivas, es necesario que éstas sean capaces de comunicarse entre ellas. Por tanto, una API (abreviatura de Application Programming Interface) son una serie de reglas que facilitan las comunicaciones entre aplicaciones. Estas aplicaciones pueden ser librerías de Python o servidores web entre otros.

Una de las principales ventajas de una API es que el solicitante no necesita saber el funcionamiento interno de la aplicación ni el lenguaje en el que esté desarrollado para poder responder y viceversa. Esto permite que diferentes servicios que usen diferentes tecnologías se comuniquen de una manera estándar.

# Introducción a FastAPI

[FastAPI](https://fastapi.tiangolo.com/) es un framework web de alto rendimiento para la construcción de APIs en Python 3.6+. 

In [ ]:
! pip install fastapi[all] pyngrok streamlit

     |████████████████████████████████| 51kB 3.3MB/s 
     |████████████████████████████████| 747kB 6.3MB/s 
     |████████████████████████████████| 7.5MB 15.9MB/s 
     |████████████████████████████████| 10.1MB 35.9MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 51kB 3.2MB/s 
     |████████████████████████████████| 645kB 19.0MB/s 
     |████████████████████████████████| 235kB 40.4MB/s 
     |████████████████████████████████| 112kB 41.4MB/s 
     |████████████████████████████████| 184kB 37.5MB/s 
     |████████████████████████████████| 61kB 4.4MB/s 
     |████████████████████████████████| 4.6MB 38.9MB/s 
     |████████████████████████████████| 112kB 42.2MB/s 
     |████████████████████████████████| 163kB 40.7MB/s 
     |████████████████████████████████| 81kB 8.8MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
     |████████████████████████████████| 3.8MB 36.6MB/s 
     |████████████████████████████████| 81kB 8.9MB/s 
     |

In [ ]:
%%writefile main.py

from fastapi import FastAPI

app = FastAPI()


@app.get("/")
async def root():
    return {"message": "Hello World"}

In [ ]:
%%writefile main.py

from typing import Optional

from fastapi import FastAPI
from pydantic import BaseModel

class Indentity(BaseModel):
    name: str
    surname: Optional[str] = None

app = FastAPI()

@app.get("/")
async def root():
    return {"message": "Hello World"}

@app.post("/testing")
async def testing(id: Indentity):
    if id.surname is None:
      message = f"Welcome to the API! My name is {id.name}"
    else: message = f"Welcome to the API! My name is {id.name} {id.surname}"
    return {"message": message}

In [ ]:
import nest_asyncio
from pyngrok import ngrok

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()

In [ ]:
! uvicorn main:app --port 8000

# ¡Manos a la obra!

Ahora vamos a comenzar con el desarrollo de la API para inferencias online. Para cada petición que recibamos de predicción vamos a realizar tres pasos:

1.   **Preprocesamiento**: en este primer paso extraeremos el dato a inferir de la petición y aplicaremos el preprocesado necesario
2.   **Inferencia**: realizaremos la inferencia sobre nuestro modelo.
3. **Postprocesado**: generaremos un JSON de respuesta con el resultado de la inferencia

**INTRODUCIR DIAGRAMA DE LA PRÁCTICA**

## Configuración de nuestro proyecto en GCP

**Los siguientes pasos es obligatorio realizarlos para seguir con la práctica.**

1.   Selecciona o crea un proyecto en GCP
2.   Asegurate de que la facturación está activada para tu proyecto.
3.   [Habilita la API de Google Cloud Storage](https://console.cloud.google.com/apis/library/storage-component.googleapis.com?q=storage).
4. [Habilita la API de Google Cloud Registry](https://console.cloud.google.com/apis/library/containerregistry.googleapis.com?q=container).
5. [Habilita la API de Google Cloud Run](https://console.cloud.google.com/apis/library/run.googleapis.com?q=cloud%20run).
6. [Habilita la API de Google Cloud Build](https://console.cloud.google.com/apis/library/cloudbuild.googleapis.com?q=cloud%20build).
7. [Habilita la API de App Engine Flexible Environment](https://console.cloud.google.com/apis/library/appengineflex.googleapis.com?q=app%20eng).
8. [Habilita la API de App Engine Admin](https://console.cloud.google.com/apis/library/appengine.googleapis.com?q=app%20engine).
9. Introduce tu ID de proyecto de GCP en la celda de abajo. Ejecuta la celda para asegurarnos de que el Cloud SDK usa el proyecto adecuado para todos los comandos en este notebook.

**Nota**: Jupyter ejecuta las lineas con el prefijo `!` como comandos shell de consola, y puede usar variables de Python en los comandos añadiendoles el prefijo `$`.

In [ ]:
PROJECT_ID = "pre-launch-keepcoding" #@param {type:"string"}
! gcloud config set project $PROJECT_ID

Updated property [core/project].


In [ ]:
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

if 'google.colab' in sys.modules:
  from google.colab import auth as google_auth
  google_auth.authenticate_user()

# If you are running this notebook locally, replace the string below with the
# path to your service account key and run this cell to authenticate your GCP
# account.
else:
  %env GOOGLE_APPLICATION_CREDENTIALS ''


## Creación bucket en Cloud Storage

**Los siguientes pasos son obligatorios.**

Cuando ejecutemos un job de entrenamiento usando el Cloud SDK, lo que hacemos es subir un paquete Python que contiene el código de entrenamiento a Google Cloud Storage. AI Platform ejecuta este paquete en el job.

Establece el nombre del bucket a continuación. El nombre tiene que ser único para todos los bucket de GCP. También tenemos que establecer la variable `REGION`, la cual usaremos para todas las operaciones a lo largo del notebook. Asegurate de [elegir una región en la que Cloud AI Platform esté disponible](https://cloud.google.com/ml-engine/docs/tensorflow/regions).

In [ ]:
BUCKET_NAME = "twitter-sentiment-keepcoding-bucket" #@param {type:"string"}
REGION = "europe-west1" #@param {type:"string"}

**Sólo si tu bucket aún no existe**: Ejecuta la siguiente celda para crear tu bucket en Cloud Storage.

In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

Finalmente, validamos que tenemos acceso al bucket de Cloud Storage mirando sus contenidos:

In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

                                 gs://twitter-sentiment-keepcoding-bucket/models/
                                 gs://twitter-sentiment-keepcoding-bucket/twitter-sentiment-batch/


## Descarga de la plantilla de código

Ahora nos descargaremos la plantilla de código que vamos a ir rellenando para el desarrollo de la práctica y establecemos el directorio como directorio de trabajo:

In [ ]:
# Clone the repository
! git clone https://gitlab.keepcoding.io/despliegue-de-algoritmos-vi/twitter-sentiment-analysis-online.git

# Set the working directory to the sample code directory
%cd ./twitter-sentiment-analysis-online

/content/twitter-sentiment-online


## Instalación de dependencias

Ejecutamos la siguiente celda para instalar las dependencias de Python necesarias para entrenar el modelo localmente y preprocesar datos. 

Cuando ejecutemos el job de entrenamiento en AI Platform, las dependencias estarán instaladas en base a la [versión del runtime](https://cloud.google.com/ml-engine/docs/tensorflow/runtime-version-list) elegido.

In [ ]:
! pip install -r requirements.txt

     |████████████████████████████████| 61kB 3.5MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 6.8MB 27.8MB/s 
     |██████████████████████▉         | 301.5MB 1.5MB/s eta 0:01:22ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 153, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 382, in run
    resolver.resolve(requirement_set)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/legacy_resolve.py", line 201, in resolve
    self._resolve_one(requirement_set, req)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/legacy_resolve.py", line 365, in _resolve_one
    abstract_dist = self._get_abstract_dist_for(req_to_install)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/legacy_

In [ ]:
# Nos aseguramos que nuestras variables de entorno no hayan desaparecido al reiniciar el kernel

print(f"Project: {PROJECT_ID}")

# Desarrollo del microservicio de inferencia

En la plantilla de código se proporciona una estructura de proyecto genérica para cualquier desarrollo de una API de inferencia online lista para ser usada de manera productiva.

El proyecto tiene la siguiente estructura:

``` bash
twitter-sentiment-online/
├── app/
│   ├── __init__.py
│   ├── api/
│   │   ├── __init__.py
│   │   └── routes/
│   │       ├── __init__.py
│   │       ├── heartbeat.py
│   │       ├── prediction.py
│   │       └── router.py
│   ├── core/
│   │   ├── __init__.py
│   │   ├── config.py
│   │   ├── enums.py
│   │   ├── event_handlers.py
│   │   └── messages.py
│   ├── main.py
│   ├── models/
│   │   ├── __init__.py
│   │   ├── heartbeat.py
│   │   ├── payload.py
│   │   └── prediction.py
│   └── services/
│       ├── __init__.py
│       └── models.py
├── Dockerfile
├── README.md
└── requirements.txt
```

En esta estructura distinguimos los siguientes componentes:

* **Dockerfile**: aquí definimos la imagen base que usaremos y como empaquetamos el proyecto.
* **requirements.txt**: especificación de dependencias a instalar en el microservicio.
* **app**: aplicación de inferencia online en FastAPI.
    * **main.py**: punto de entrada para la ejecución de la aplicación.
    * **models**: en este encontramos la definición de los esquemas que usaremos dentro de la aplicación.
    * **services**: en este módulo incluiremos la implementación de nuestra clase de inferencia.
    * **api/routes**: módulo en el que definiremos los diferentes endpoints que tendrá la API.
    * **api/core**: módulo donde estarán funcionalidades comunes al servicio y configuraciones.

In [ ]:
%cd /content/twitter-sentiment-analysis-online/

In [ ]:
import os

os.environ["DEFAULT_MODEL_PATH"] = "/content/twitter-sentiment-analysis-online/"

In [ ]:
! gsutil -m cp \
  "gs://$BUCKET_NAME/twitter-sentiment-batch/data/model/model.h5" \
  "gs://$BUCKET_NAME/twitter-sentiment-batch/data/model/tokenizer.pkl" \
  .

Para poder probar el correcto funcionamiento del servicio en local:

In [ ]:
import nest_asyncio
from pyngrok import ngrok

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()

In [ ]:
! uvicorn app.main:app --port 8000

## Creando la interfaz usando Streamlit

[Streamlit](https://www.streamlit.io/) es un framework para la creación de Webapps orientado a datos e Inteligencia Artificial basado en Python.

En este caso vamos a desarrollar un pequeño frontal para poder invocar nuestro recién desarrollado servicio de inferencia para realizar predicciones.

In [ ]:
%mkdir /content/prediction-front

In [ ]:
%cd /content/prediction-front

In [ ]:
%%writefile front.py

import requests
import validators
import streamlit as st
import pandas as pd


st.title("Sentiment Analysis Predictions")

st.markdown(
    "Welcome! With this app you can predict the sentiment of a given text using Deep Learning :smile:"
)

st.write("Fist, paste below the predictor server URL: ")

server_url = st.text_input("Server URL", value="")

if server_url != "":
    st.write(f"Server URL: {server_url}")


In [ ]:
import nest_asyncio
from pyngrok import ngrok

ngrok_tunnel = ngrok.connect(8501)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()

In [ ]:
! streamlit run front.py

# Despliegue en GCP de la aplicación

Ahora que ya tenemos nuestro servicio de inferencia online funcionando, estamos listos para desplegar a escala para que soporte miles de peticiones de manera concurrente, para ello utilizaremos el servicio de GCP Cloud Run.

Cloud Run es un servicio Serverless (o sin servidor). Serveless nos facilita la puesta en producción de aplicaciones pues, en este caso GCP, se encarga de gestionar la infraestructura y recursos desplegados para nuestra aplicación en base a la carga que tenga esta misma a lo largo del tiempo. 

Esta práctica nos permita escalar de manera casi infinita, desde dar servicio desde tan solo a decenas de usuarios como a millones de manera concurrente sin tener que realizar ningún ajuste.

Las aplicaciones Serverless son una alternativa a los microservicios y los monolitos.

## Creando una aplicacion Serverless

Para crear nuestra aplicación serverless tan solo nos tenemos que preocupar de que nuestro código funciona y empaquetar todo en una imagen Docker que, finalmente, será lo que despleguemos Cloud Run.

Para crear esta imagen, dado que estamos en un entorno de Google Colab no podemos usar Docker, haremos uso del servicio Cloud Build en GCP, que se encargará de generarnos la imagen con nuestro Dockerfile de la aplicación y finalmente la guardará en el Google Container Registry, donde se almacenan las imágenes Docker.

In [ ]:
%cd /content/twitter-sentiment-analysis-online/

In [ ]:
! gcloud builds submit --tag gcr.io/$PROJECT_ID/sentiment-analysis-server

Finalmente, procederemos a desplegar la imagen docker en el servicio de GCP Cloud Run

## Probando nuestra aplicación desplegada

Una vez desplegado nuestra aplicación serverless de inferencia online... ¡Podemos usarla desde cualquier lugar del planeta! Tanto si es un usuario como si son millones. Para hacer un ejemplo de petición a nuestro modelo desplegado, podemos hacer lo siguiente:

In [ ]:
! curl -X POST "https://predict-service-p26tpbhusq-uc.a.run.app/api/model/predict" -H  "accept: application/json" -H  "Content-Type: application/json" -d "{\"text\":\"i hate\"}"

## Despliegue de la interfaz

Para desplegar la interfaz de nuestra aplicación usaremos App Engine. Lo único que necesitaremos será un Dockerfile con el cual construiremos una imagen que será la que desplegaremos.

In [ ]:
%cd /content/prediction-front

In [ ]:
%%writefile requirements.txt

requests
validators
pandas
streamlit

In [ ]:
%%writefile Dockerfile

FROM python:3.7.8-slim

# remember to expose the port your app'll be exposed on.
EXPOSE 8080

RUN pip install -U pip

COPY requirements.txt app/requirements.txt
RUN pip install -r app/requirements.txt

# copy into a directory of its own (so it isn't in the toplevel dir)
COPY front.py /app/front.py
WORKDIR /app

# run it!
ENTRYPOINT ["streamlit", "run", "front.py", "--server.port=8080", "--server.address=0.0.0.0"]

In [ ]:
%%writefile app.yaml

runtime: custom
env: flex
# service: sentiment-analysis-front

In [ ]:
! gcloud app deploy app.yaml